#### Sensitivity analysis:

Imagine a portfolio of securities whose value can be approximately represented by a single risk factor x (random variable), as follows: ppv=f(x) = k1 * e^(-T1*x),
where k1 and T1 are constants. The portfolio owner wants to add a new security to the portfolio, which has the generic form g(x, k2, T2), where x is the same
random variable. The constants k1 and k2 are the nominal values of the securities. Create a function that returns four values: the value of the old portfolio, the
value of the new portfolio, the sensitivity of the old portfolio to factor x, and the sensitivity of the new portfolio to factor x taking into account the weighted face
value (positive for long positions, negative for short positions). The inputs to the function should be k1, T1, x and g(x, k2, T2):

In [15]:
import numpy as np
import math
from scipy.optimize import minimize

In [5]:
def portfolio_sensitivity(k1, T1, x, g, k2, T2): #inputs function
    old_portfolio_value = k1 * np.exp(-T1 * x) # Valor aprox antiguo portf. con formula proporcionada
    new_security_value = g(x, k2, T2)     # Valor nueva security
    new_portfolio_value = old_portfolio_value + new_security_value  # Valor nuevo portafolio cuando añadimos new security.
                                                                    # Asumimos que es correcto añadir el valor security nueva al portafolio existente.
    sensitivity_old_portfolio = -k1 * T1 * np.exp(-T1 * x) # Sensibilidad del antiguo portafolio a factor x (cambia x, k1 y T1 cte). Usamos derivada parcial.
    sensitivity_new_portfolio = (new_portfolio_value - old_portfolio_value) / x  # Sensibilidad del nuevo portafolio al añadir la nueva security,en función del factor x. 
                                                                                 # También podriamos suponer que k2 es el valor nominal de las posiciones largas (positivas).
                                                                                 # y que k1 es el valor nominal de las posiciones cortas (negativas).
                                                                                 # En este caso quedaría: sensitivity_new_portfolio = (new_portfolio_value - old_portfolio_value) / (k2 - k1)
    
    return old_portfolio_value, new_portfolio_value, sensitivity_old_portfolio, sensitivity_new_portfolio

Tomamos la derivada parcial respecto a x. Así sabemos cuanto cambia el portfolio cuando cambia x manteniendo 
las demas variables constantes. Esto es una simplificación, ya que se asume que as relaciones entre los activos en  el portfolio y el factor de riesgo x son lineales, implicando que cambios en x conducen a cambios proporcionales en el valor de los activos del portfolio. Tampoco se tienen en cuenta momentos de alta volatilidad o eventos extremos del mercado y la posible correlación dinámica entre activos del portfolio.

"Análisis de sensibilidad (estadística)." Wikipedia. Wikipedia, la enciclopedia libre, 28 de septiembre de 2023. https://es.wikipedia.org/wiki/An%C3%A1lisis_de_sensibilidad_(estad%C3%ADstica).

Para la sensiblidad del nuevo portfolio, se observa cómo cambia su valor total (incluiendo la nueva security)en relación con cambios en x.

#### Optimization: 

Imagine the portfolio owner wants to offset the risk associated with x in the new portfolio. To achieve this, the T2 parameter,
instead of being a constant, can be adjusted. Develop a routine that requires inputs such as k1, T1, x, and g(x, k2, t2) to identify
the potential values of t2 that offset all risk associated with x. Use any optimization techniques as needed.

In [12]:
def risk_to_x_function(T2, k1, T1, x, k2): # Función que calcule el riesgo asociado a x en el nuevo portfolio
    #_, _, _, sensitivity_new_portfolio = portfolio_sensitivity(k1, T1, x, g, k2, T2)
    _, _, _, sensitivity_new_portfolio = portfolio_sensitivity(k1, T1, x, lambda x, k2, T2: g(x, k2, T2), k2, T2)  # Extraes de la función portfolio_sensitivity, la sensibilidad del nuevo portfolio
    risk_to_x = abs(sensitivity_new_portfolio)     # riesgo asociado a x nuevo portfolio == sensibilidad al factor x (delta) en abs value.
    
    return risk_to_x 

In [13]:
def offset_risk(k1, T1, x, k2): # Función para valor óptimo de T2 que minimiza el riesgo de x. (Estamos buscando un solo valor óptimo de T2.)
    initial_guess = 1.0      # Valor inicial para T2, suposición y punto de partida
    constraint = {'type': 'eq', 'fun': lambda T2: risk_to_x_function(T2, k1, T1, x, k2)} # restricción optimización, riesgo == 0 
    result = minimize(lambda T2: 0, initial_guess, constraints=constraint)     # Realiza la optimización para que riesgo sea 0.
    optimized_T2 = result.x[0]     # Extrae el (primer) valor óptimo de T2 para que el resultado del riesgo sea 0.
    
    return optimized_T2

Como una simplificación, tomamos la sensibilidad como una medida de riesgo, dado que el riesgo se relaciona directamente con la variación del valor de la cartera debido a cambios en x. Además solo se conoce el valor total del antiguo portafolio y no hay información detallada sobre los activos individuales que lo componen. 


Se utiliza la biblioteca scipy y el método de optimización minimize, que se utiliza para encontrar los valores óptimos de las variables de ajuste (en este caso, T2) que minimizan una función dada (risk_to_x_function).
La restricción está expresada como un diccionario (constraint). La clave 'type' en 'eq', define una constraint de igualdad. 'fun' da la función que calcula el riesgo (lambda, T2: risk_to_x_function(T2, k1, T1, x, k2)). Esto significa que la optimización intentará encontrar un valor de T2 de manera que el resultado de risk_to_x_function(T2, k1, T1, x, k2) sea cero. (lambda T2: 0 )

SciPy Community. (2023). SciPy.optimize.minimize. https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.htm

#### Stochastic analysis: 

Imagine that x is a random variable that follows a simple stochastic process, the Wiener process: dx = x * q * dW. Build a
function that calculates the value of the new portfolio, taking as inputs: k1, T1, g(x, k2, T2), q, deltaStepOfWiener, and
numMaxFinalSteps.

In [17]:
def stochastic_portfolio_value(k1, T1, g, k2, T2, q, deltaStepOfWiener, numMaxFinalSteps):
    x = 0.0 # Inicializa, para tener valor inicial de x
    new_portfolio_value = 0.0  # Inicializa, para tener un valor inicial del nuevo portafolio
   
    for step in range(numMaxFinalSteps): #bucle for, step con valores entre 0 y (numMaxFinalSteps - 1)
        dW = np.random.normal(0, np.sqrt(deltaStepOfWiener)) # Crea incremento aleatorio con distribució normal, representando naturaleza estocástica del proceso. 
                                                             # 0 media (propiedad proceso wiener) y st. deviation para varianza del incremento proporcional al intervalodel tiempo
        x += x * q * dW # Actualiza el valor de x en cada iteracion
        new_security_value = g(x, k2, T2)   # Calcula el valor del nuevo activo
        new_portfolio_value += new_security_value  # Actualiza el valor portafolio, sumándole el valor del nuevo activo
    
    return new_portfolio_value



        

La principal característica del proceso de Wiener es que los incrementos son independientes entre sí y están distribuidos normalmente con una media de cero y una varianza proporcional al tamaño del incremento de tiempo. Al configurar la media en 0, estamos modelando la propiedad de Wiener.

"Proceso de Wiener (Movimiento Browniano)." Wikipedia. Wikipedia, la enciclopedia libre, 28 de septiembre de 2023. 
https://es.wikipedia.org/wiki/Proceso_de_Wiener#:~:text=En%20matem%C3%A1ticas%2C%20un%20proceso%20de,en%20honor%20a%20Robert%20Brown.